In [ ]:
# Import necessary libraries
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install necessary packages
!pip install tensorflow==2.12
!pip install keras==2.12

In [ ]:
# Import necessary libraries
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
from os import listdir
import cv2
import matplotlib.pyplot as plt
import imutils
from PIL import Image
from sklearn.model_selection import train_test_split
from keras import layers
from keras.utils.vis_utils import plot_model
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  *
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.applications import DenseNet121, VGG19, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

import PIL.Image
import matplotlib.pyplot as mpimg
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.preprocessing import image

In [ ]:
# Install imutils library
!pip install imutils

In [ ]:
# List the files in the directory
os.listdir('/content/drive/MyDrive/Colab Notebooks/Project Implementation/RIM ONE DL_Data cleaning/RIM ONE DL _after_downsampling')

In [ ]:
#showing image of normal
im = Image.open('/content/drive/MyDrive/Colab Notebooks/Project Implementation/RIM ONE DL_Data cleaning/RIM ONE DL _after_downsampling/normal/result_r1_Im003.png').resize((255, 255))
im

In [ ]:
#showing image of glaucoma
im = Image.open('/content/drive/MyDrive/Colab Notebooks/Project Implementation/RIM ONE DL_Data cleaning/RIM ONE DL _after_downsampling/glaucoma/result_r1_Im118.png').resize((255, 255))
im

In [ ]:
#setting the directory of our data
glaucoma = os.listdir('/content/drive/MyDrive/Colab Notebooks/Project Implementation/RIM ONE DL_Data cleaning/RIM ONE DL _after_downsampling/glaucoma')
normal = os.listdir('/content/drive/MyDrive/Colab Notebooks/Project Implementation/RIM ONE DL_Data cleaning/RIM ONE DL _after_downsampling/normal')

#concate two data
data = np.concatenate([glaucoma,normal])
assert (len(data) == len(glaucoma)+len(normal))

#mapping our classes to 1 and 0. 1 == glaucoma and 0 == normal
target_x = np.full(len(glaucoma),1)
target_y = np.full(len(normal),0)

#label of our data
data_target = np.concatenate([target_x,target_y])

assert (len(data_target) == len(target_x)+len(target_y))
assert (len(data_target) == len(data))

print(len(target_x), len(target_y))

In [ ]:
#Setting the height and width of the image for our model
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMAGE_SIZE = [IMG_HEIGHT, IMG_WIDTH]

In [ ]:

X_data = []

# Reading images from the glaucoma directory, resizing, preprocessing, and appending to the list
for file in glaucoma:
    img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Project Implementation/RIM ONE DL_Data cleaning/RIM ONE DL _after_downsampling/glaucoma/' + file)
    img2 = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    #preprocessed_img = preprocess_image(img2)
    #X_data.append(preprocessed_img)
    X_data.append(img2)

# Reading images from the normal directory, resizing, preprocessing, and appending to the list
for file in normal:
    img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Project Implementation/RIM ONE DL_Data cleaning/RIM ONE DL _after_downsampling/normal/' + file)
    img2 = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    #preprocessed_img = preprocess_image(img2)
    #X_data.append(preprocessed_img)
    X_data.append(img2)

X_data = np.squeeze(X_data)
assert (len(X_data) == len(data) == len(data_target))


In [ ]:
# normalize data
X = X_data.astype('float32')
X /= 255

In [ ]:
X_data.shape

In [ ]:
# split the data into 75:15:15 ratio
X_train, X_test, y_train, y_test = train_test_split(X, data_target, random_state=50, test_size=0.15, stratify = data_target)

print(len(y_train))
print(len(y_test))

In [ ]:
#I add
#sample of training data
import matplotlib.pyplot as plt
num_samples = 5
fig, axs = plt.subplots(1, num_samples, figsize=(10, 4))

for i in range(num_samples):
    axs[i].imshow(cv2.cvtColor(X_train[i], cv2.COLOR_BGR2RGB))  # Assuming grayscale images
    axs[i].set_title(''.format(i))
    axs[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#from tensorflow.keras.applications import DenseNet121, VGG19, ResNet50
#define input shape
input_shape = (224,224,3)#DenseNet121, VGG19, ResNet50

#get the pretrained model
base_model = tf.keras.applications.ResNet50(input_shape= input_shape,
                                               include_top=False,
                                               weights='imagenet')

#set the trainable method of covolution layer as false
# why set to false?? because we don't want to mess up the pretrained weights of the model!!
base_model.trainable = False

In [ ]:
# Initialize a sequential model
model = Sequential()

# Add the base model to the sequential model
model.add(base_model)

# Add a batch normalization layer to the sequential model
model.add(BatchNormalization())

# Add a dropout layer with rate 0.3 to the sequential model
model.add(Dropout(0.3))

# Add a dense layer with 256 units and ReLU activation function to the sequential model
model.add(Dense(256, activation='relu'))

# Add a flatten layer to the sequential model
model.add(Flatten())

# Add a dropout layer with rate 0.5 to the sequential model
model.add(Dropout(0.5))

# Add a dense layer with 128 units and ReLU activation function to the sequential model
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()


In [ ]:
from keras.optimizers import Adam

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),#lr=0.01
    metrics=["acc"]
)

In [ ]:
#checkpoint which stores the weights of the model when maximum validation accuracy is found
checkpoint_filepath = "/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/checkpoint_RIM-ONE DL/Classification_DenseNet121_RIM-ONE DL_15_batch_8_epoch_weights.best.hdf5"

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)



history = model.fit(X_train, y_train, batch_size = 8,
                    epochs = 15, validation_split = 0.18, callbacks=[checkpoint])

In [ ]:
# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/Saved_model_RIM-ONE_DL/Classification_DenseNet121_RIM-ONE DL_epoch_15_batch_8_lr=0.01_model.h5')

In [ ]:
history = model.history.history

train_loss = history['loss']
val_loss = history['val_loss']
train_acc = history['acc']
val_acc = history['val_acc']


# Loss
plt.figure()
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/graphes_RIM-ONE_DL/Classification_ResNet50_RIM-ONE DL_15_batch_8_epoch_lr=0.01_loss_graph.png')#add
plt.show()

# Accuracy#
plt.figure()
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/graphes_RIM-ONE_DL/Classification_ResNet50_RIM-ONE DL_15_batch_8_epoch_lr=0.01_accuracy_graph.png')
plt.show()

In [ ]:
import pickle

# Save the history
with open('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/saved history/Classification_ResNet50_epoch_15_batch_8_lr=0.01_history.pickle', 'wb') as file:
    pickle.dump(history, file)

# **Model Evaluation**

In [ ]:
history = model.history.history

train_loss = history['loss']
val_loss = history['val_loss']
train_acc = history['acc']
val_acc = history['val_acc']


# Loss
plt.figure()
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/graphes_RIM-ONE_DL/Classification_ResNet50_RIM-ONE DL_15_batch_8_epoch_lr=0.01_loss_graph.png')#add
plt.show()

# Accuracy#
plt.figure()
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/graphes_RIM-ONE_DL/Classification_ResNet50_RIM-ONE DL_15_batch_8_epoch_lr=0.01_accuracy_graph.png')
plt.show()

In [ ]:
import pickle

# Save the history
with open('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/saved history/Classification_ResNet50_epoch_15_batch_8_lr=0.01_history.pickle', 'wb') as file:
    pickle.dump(history, file)

In [ ]:
# Evaluate the model on test set
score = model.evaluate(X_test, y_test, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])

In [ ]:
y_hat = model.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_hat]

#metrics calculation

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)

# ROC AUC
auc = roc_auc_score(y_test, y_pred)
print('ROC AUC: %f' % auc)

# confusion matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

# specificity calculation
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)
print('Specificity: ',specificity)

#classification report
print(classification_report(y_test, y_pred))

#To save

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Convert report to a DataFrame for visualization
df_report = pd.DataFrame(report).transpose()

# Plot classification report
plt.figure(figsize=(8, 6))
sns.heatmap(df_report.iloc[:-1, :].T, annot=True, cmap='Blues')
plt.xlabel('Metrics')
plt.ylabel('Classes')
plt.title('Classification Report')

# Save classification report as image
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/graphes_RIM-ONE_DL/Classification_ResNet50_RIM-ONE DL_epoch_15_early_14_batch_8_lr=0.001_classification_report.png')


In [ ]:
# plot confusion matrix
categories = ['Normal', 'Glaucoma']
df_matrix = pd.DataFrame(matrix, index = categories, columns = categories)
sns.heatmap(df_matrix, annot=True, cmap='Blues', fmt='d')
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/graphes_RIM-ONE_DL/ClassificationResNet50_RIM-ONE DL_epoch_15_early_14_batch_8_lr=0.001_confusion_matrix.png')

In [ ]:
#ROC Curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='ResNet50_')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/Project Implementation/Classification/graphes_RIM-ONE_DL/Classification_ResNet50_RIM-ONE DL_epoch_15_early_14_batch_8_lr=0.001_ROC Curve_graph.png')
plt.show();

In [ ]:
# Evaluate the model on test set

#Check the confusion matrix for various thresholds. Which one is good?
#Need to balance positive, negative, false positive and false negative.
#ROC can help identify the right threshold.
##################################################################
"""
Receiver Operating Characteristic (ROC) Curve is a plot that helps us
visualize the performance of a binary classifier when the threshold is varied.
"""
#ROC
from sklearn.metrics import roc_curve
y_preds = model.predict(X_test).ravel()

fpr, tpr, thresholds = roc_curve(y_test, y_preds)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'y--')
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.show()



In [ ]:
# Evaluate the model on test set

#AUC
#Area under the curve (AUC) for ROC plot can be used to understand hpw well a classifier
#is performing.
#% chance that the model can distinguish between positive and negative classes.

from sklearn.metrics import auc
auc_value = auc(fpr, tpr)
print("Area under curve, AUC = ", auc_value)